# Setup

## Parser Setup

In [1]:
import configparser

file_path = 'config.ini'
config = configparser.ConfigParser()
config.read(file_path)

['config.ini']

## Database and Integratiy Setup

In [2]:
import os

# Change to the parent directory to access the DFVC module
os.chdir('..')
from src import DFVC

# Define constant for database access
DB_DIR = 'data'

# Define constants for raw data acess
RAW_DATA_FILENAME = config['database']['raw_data_filename']
RAW_HASH = config['database']['raw_data_version_hash']

RAW_DATA_PATH = os.path.join(DB_DIR, RAW_DATA_FILENAME)

# Define constants for raw data processing (preprocessing step 1)
PRP_S1_DF_NAME = config['database']['prp_data_s1_df_name']
PRP_S1_FILENAME = config['database']['prp_data_s1_filename']
PRP_S1_HASH = config['database']['prp_data_s1_version_hash']

PRP_S1_PATH = os.path.join(DB_DIR, PRP_S1_FILENAME)

## Data Manipulation Setup

In [3]:
import pandas as pd
from IPython.display import Image, display

# Data Integrity Verification and Getting Started with the Data

In [ ]:
raw_dfvc = DFVC.load_dfvc_file(RAW_DATA_PATH)
raw_dfvc.compare_versions(RAW_HASH)
display(raw_dfvc)

Version integrity verified successfully.


In [5]:
raw = raw_dfvc.get_dataframe()
raw.head()

,iframe_title,timeseries_category,timeseries_value
0,Instagram revenues - Infogram,Q1 2015,111
1,Instagram revenues - Infogram,Q2 2015,139
2,Instagram revenues - Infogram,Q3 2015,141
3,Instagram revenues - Infogram,Q4 2015,201
4,Instagram revenues - Infogram,Q1 2016,261


# Data Preprocessing/Preparation (Stage 1)

## About


This step aims to prepare the data for exploratory analysis and predictive modeling, transforming them into time series and organizing categorical and numerical columns. The raw data, from different sources, will be unified to ensure consistency, facilitate analysis, and filtered to retain only the last five years, meeting the research objective.

### 1. Intermediary DataFrame Creation

- Initialize an intermediate DataFrame to store the data throughout the preparation steps.
- Create the columns/variables of the final DataFrame initially empty, which will be filled during the processing stages. Columns to be created:
  - `quarter`: Time series.
  - `quarter_label`: Label for each time series, in this case, quarters.
  - `ig_maus`: Monthly active users for Instagram.
  - `ig_revs`: Revenue for Instagram.
  - `tk_maus`: Monthly active users for TikTok.
  - `tk_revs`: Revenue for TikTok.

### 2. Time Series Filling

- Fill the columns with the values for each day of the quarter, using the datetime format to represent closing dates.

### 3. Numerical Value Filling

- Insert the numerical values into the columns corresponding to MAUs (monthly active users) and revenue.

### 4. Conversion to Appropriate Numeric Types

- Convert the MAUs and revenue columns to the appropriate numeric types to ensure consistency and facilitate further analysis.





## Intermediary DataFrame Creation

In [6]:
def create_inter_df() -> pd.DataFrame:
    return pd.DataFrame(columns=['quarter', 'quarter_label', 'ig_maus', 'ig_revs', 'tk_maus', 'tk_revs'])

# Create the intermediary DataFrame
prp_s1_inter = create_inter_df()
prp_s1_inter

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs


## Time Series Filling

In [7]:
def fill_quarter_label(input_df, output_df, quarter_label_col='quarter_label', raw_timeseries_category='timeseries_category') -> pd.DataFrame:
    output_df[quarter_label_col] = input_df[raw_timeseries_category].unique()
    return output_df

# Apply the transformation to the intermediary DataFrame
prp_s1_inter = fill_quarter_label(input_df=raw, output_df=prp_s1_inter)
display(prp_s1_inter.head())
display(prp_s1_inter.tail())

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
0,NaN,Q1 2015,NaN,NaN,NaN,NaN
1,NaN,Q2 2015,NaN,NaN,NaN,NaN
2,NaN,Q3 2015,NaN,NaN,NaN,NaN
3,NaN,Q4 2015,NaN,NaN,NaN,NaN
4,NaN,Q1 2016,NaN,NaN,NaN,NaN


,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
42,NaN,Q4 2013,NaN,NaN,NaN,NaN
43,NaN,Q1 2014,NaN,NaN,NaN,NaN
44,NaN,Q2 2014,NaN,NaN,NaN,NaN
45,NaN,Q3 2014,NaN,NaN,NaN,NaN
46,NaN,Q4 2014,NaN,NaN,NaN,NaN


In [8]:
def fill_quarter(output_df, quarter_label_col='quarter_label', quarter_col='quarter', last_years_to_keep=5) -> pd.DataFrame:
    def convert_to_date(quarter):
        q, year = quarter.split()
        q = int(q[1])
        month, day = {1: (3, 31), 2: (6, 30), 3: (9, 30), 4: (12, 31)}.get(q, (1, 1))
        return pd.Timestamp(year=int(year), month=month, day=day)

    # Apply the transformation to create the 'quarter' column
    output_df[quarter_col] = output_df[quarter_label_col].apply(convert_to_date)

    # Sort values by 'quarter' and reset the index
    output_df = output_df.sort_values(by=quarter_col).reset_index(drop=True)

    # Define the current year and filter data to keep only the last 'last_years_to_keep' years
    current_year = pd.Timestamp.now().year
    output_df = output_df[output_df[quarter_col].dt.year >= (current_year - last_years_to_keep)].reset_index(drop=True)

    return output_df

# Apply the transformation to the intermediary DataFrame
prp_s1_inter = fill_quarter(prp_s1_inter)

display(prp_s1_inter.head())
display(prp_s1_inter.tail())

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
0,2019-03-31,Q1 2019,NaN,NaN,NaN,NaN
1,2019-06-30,Q2 2019,NaN,NaN,NaN,NaN
2,2019-09-30,Q3 2019,NaN,NaN,NaN,NaN
3,2019-12-31,Q4 2019,NaN,NaN,NaN,NaN
4,2020-03-31,Q1 2020,NaN,NaN,NaN,NaN


,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
18,2023-09-30,Q3 2023,NaN,NaN,NaN,NaN
19,2023-12-31,Q4 2023,NaN,NaN,NaN,NaN
20,2024-03-31,Q1 2024,NaN,NaN,NaN,NaN
21,2024-06-30,Q2 2024,NaN,NaN,NaN,NaN
22,2024-09-30,Q3 2024,NaN,NaN,NaN,NaN


## Numerical Value Filling

In [9]:
def fill_maus_and_revs(input_df, output_df) -> pd.DataFrame:
    for _, row in input_df.iterrows():
        platform = 'ig' if 'Instagram' in row['iframe_title'] else 'tk' if 'TikTok' in row['iframe_title'] else None
        if platform:
            if 'monthly app users' in row['iframe_title'] or 'MAUs' in row['iframe_title']:
                output_df.loc[output_df['quarter_label'] == row['timeseries_category'], f'{platform}_maus'] = row['timeseries_value']
            elif 'revenues' in row['iframe_title']:
                output_df.loc[output_df['quarter_label'] == row['timeseries_category'], f'{platform}_revs'] = row['timeseries_value']
    return output_df

# Apply the transformation to the intermediary DataFrame
prp_s1_inter = fill_maus_and_revs(raw, prp_s1_inter)

display(prp_s1_inter.head())
display(prp_s1_inter.tail())

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
0,2019-03-31,Q1 2019,1160,3255,333,56
1,2019-06-30,Q2 2019,1210,3887,381,65
2,2019-09-30,Q3 2019,1255,4066,439,82
3,2019-12-31,Q4 2019,1300,5016,508,167
4,2020-03-31,Q1 2020,1365,4475,583,347


,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
18,2023-09-30,Q3 2023,2145,12736,1616,4241
19,2023-12-31,Q4 2023,2180,15322,1635,4997
20,2024-03-31,Q1 2024,2231,14472,1666,5855
21,2024-06-30,Q2 2024,2250,15638,1685,6922
22,2024-09-30,Q3 2024,2270,16641,1726,7737


## Conversion to Appropriate Numeric Types

In [10]:
def convert_maus_and_revs(output_df, columns_to_convert: list = ['ig_maus', 'ig_revs', 'tk_maus', 'tk_revs']) -> pd.DataFrame:
    for col in columns_to_convert:
        output_df[col] = pd.to_numeric(output_df[col], errors='coerce')
    return output_df

# Apply the transformation to the intermediary DataFrame
prp_s1_inter = convert_maus_and_revs(prp_s1_inter)

display(prp_s1_inter.head())
display(prp_s1_inter.tail())

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
0,2019-03-31,Q1 2019,1160,3255,333,56
1,2019-06-30,Q2 2019,1210,3887,381,65
2,2019-09-30,Q3 2019,1255,4066,439,82
3,2019-12-31,Q4 2019,1300,5016,508,167
4,2020-03-31,Q1 2020,1365,4475,583,347


,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
18,2023-09-30,Q3 2023,2145,12736,1616,4241
19,2023-12-31,Q4 2023,2180,15322,1635,4997
20,2024-03-31,Q1 2024,2231,14472,1666,5855
21,2024-06-30,Q2 2024,2250,15638,1685,6922
22,2024-09-30,Q3 2024,2270,16641,1726,7737


## Data and Types Verification

In [11]:
display(prp_s1_inter)
display(prp_s1_inter.dtypes)

,quarter,quarter_label,ig_maus,ig_revs,tk_maus,tk_revs
0,2019-03-31,Q1 2019,1160,3255,333,56
1,2019-06-30,Q2 2019,1210,3887,381,65
2,2019-09-30,Q3 2019,1255,4066,439,82
3,2019-12-31,Q4 2019,1300,5016,508,167
4,2020-03-31,Q1 2020,1365,4475,583,347
5,2020-06-30,Q2 2020,1435,4888,700,638
6,2020-09-30,Q3 2020,1490,5399,667,778
7,2020-12-31,Q4 2020,1555,7075,756,883
8,2021-03-31,Q1 2021,1630,6830,812,829
9,2021-06-30,Q2 2021,1690,7792,902,1114


quarter          datetime64[ns]
quarter_label            object
ig_maus                   int64
ig_revs                   int64
tk_maus                   int64
tk_revs                   int64
dtype: object

# Data Exportation and Version/Integrity Verification

In [12]:
# Data preparation to export - Initialize the DFVC object with the prp s1 data
prp_s1 = DFVC(prp_s1_inter, PRP_S1_DF_NAME)
display(prp_s1)

# Integrity verification
prp_s1.compare_versions(PRP_S1_HASH)

# At least, prp_s1 data exportation
prp_s1.export_as_dfvc_file(PRP_S1_PATH)

Version integrity verified successfully.
DFVC object successfully saved to data/2_prp_s1.dfvc.
